# Tensor基础知识

Tensor是pytorch中一种基本的数据结构，它与numpy中的ndarray非常相似，通过将数据组织成多维张量的方式来对模型的输入、输出，以及模型参数等进行编码。使用Tensor表示的输入、输出数据以及模型参数，可以在GPU等硬件上获得计算加速，并且可以使用pytorch中的Auto-Grad（自动求导）等机制进行模型的训练。

In [2]:
import torch
import numpy as np

### Tensor的创建


Tensor的创建方式主要有四种：
- 直接从数据创建

In [5]:
data = [[0, 1],[2, 3]]
x = torch.tensor(data, dtype=torch.float32, device='cpu')
x_auto = torch.tensor(data) # 如果不指定数据类型以及所在设备，pytorch会自动推断数据类型，并且默认使用cpu

- 从Numpy创建

In [6]:
array = np.array(data)
x = torch.from_numpy(array)
x_numpy = x.numpy() # 使用.numpy()操作将tensor转化为numpy的ndarray格式

- 从其他tensor创建

In [14]:
x_ones = torch.ones_like(x)
x_zeros = torch.zeros_like(x)
x_rand = torch.rand_like(x, dtype=torch.float64) #可以修改tensor的数据类型，如果不显式指定则保留原先的数据类型

- 创建指定特殊类型的tensor

In [9]:
ones = torch.ones(2, 3)
zeros = torch.zeros(3, 4, dtype=torch.float32, device='cpu')
eye = torch.eye(3)
rand = torch.rand(2, 3)

### Tensor的属性

Tensor具有数据类型（dtype），所在设备（device），形状（shape）等属性，根据一定的原则可以对属性进行修改。

In [10]:
tensor = torch.rand(3, 4)
print(f'Shape of tensor: {tensor.shape}')
print(f'Dtype of tensor: {tensor.dtype}')
print(f'Device of tensor: {tensor.device}')

Shape of tensor: torch.Size([3, 4])
Dtype of tensor: torch.float32
Device of tensor: cpu


- 修改tensor的形状可以通过调用tensor.view()或者tensor.reshape()来实现，注意形状改变不可以改变数据量。

In [11]:
tensor_view = tensor.view(1, 12)
tensor_reshape = tensor.reshape(4, 3)
print(f'Shape of tensor_view: {tensor_view.shape}')
print(f'Shape of tensor_reshape: {tensor_reshape.shape}')

Shape of tensor_view: torch.Size([1, 12])
Shape of tensor_reshape: torch.Size([4, 3])


- 如果想要增删、复制某个维度，可以使用tensor.squeeze()、tensor.unsqueeze()、tensor.repeat()等函数。

In [6]:
tensor = torch.randn(1, 3, 4)
tensor_squeeze = tensor.squeeze(0)
print(f'Shape of tensor_squeeze: {tensor_squeeze.shape}')
tensor_unsqueeze = tensor.unsqueeze(-1)
print(f'Shape of tensor_unsqueeze: {tensor_unsqueeze.shape}')
tensor_repeat = tensor.repeat(2, 1, 1)
print(f'Shape of tensor_repeat: {tensor_repeat.shape}')


Shape of tensor_squeeze: torch.Size([3, 4])
Shape of tensor_unsqueeze: torch.Size([1, 3, 4, 1])
Shape of tensor_repeat: torch.Size([2, 3, 4])


- 修改tensor的数据类型可以通过调用tensor.float(),tensor.double(),tensor.long()等命令，或者使用torch.type()并传入目标数据类型来实现。

In [12]:
tensor_float = tensor.float()
tensor_double = tensor.double()
tensor_int32 = tensor.type(torch.int32)
print(f'Dtype of tensor_float: {tensor_float.dtype}')
print(f'Dtype of tensor_double: {tensor_double.dtype}')
print(f'Dtype of tensor_int32: {tensor_int32.dtype}')

Dtype of tensor_float: torch.float32
Dtype of tensor_double: torch.float64
Dtype of tensor_int32: torch.int32


- 修改tensor所在的设备可以通过调用tensor.to()并传入设备名称来实现，或者直接使用tensor.cuda()/tensor.cpu()来将tensor在CPU和GPU之间搬运。

In [13]:
if torch.cuda.is_available():
    tensor_gpu0 = tensor.to('cuda:0')
    tensor_gpu = tensor.cuda()
    print(f'Device of tensor_gpu0: {tensor_gpu0.device}')
    print(f'Device of tensor_gpu: {tensor_gpu.device}')
tensor_cpu = tensor.cpu()
print(f'Device of tensor_cpu: {tensor_cpu.device}')

Device of tensor_cpu: cpu


### Tensor的运算

Pytorch为Tensor实现了丰富的运算操作，包括索引、切片、合并、转置、数学运算等操作，很多操作与Numpy中对ndarray的操作非常相似，这些操作都可以在GPU上运行以获得运算加速。

- 索引、切片:
可以对tensor的每一维度索引若干长度$s_1,s_2,s_3,...$，得到形状为$(s_1, s_2, s_3, ...)$的一个新tensor。

In [9]:
tensor = torch.randn(3, 4, 5).float()
print(tensor[1:3, 2:3, 0:2].shape)
print(tensor[1:3, 2, 0:2].shape) # 如果对某个维度的索引为某个指定的位置，那么得到的新tensor将失去该维度。
print(tensor[:, :3, :4].shape)

torch.Size([2, 1, 2])
torch.Size([2, 2])
torch.Size([3, 3, 4])


- 合并：使用torch.cat()或torch.stack()对一组tensor进行合并。对于torch.cat来说，要求被合并的一组tensor除了在被合并的维度上以外，具有相同的大小。对于torch.stack来说，要求被合并的一组tensor在所有维度上都具有相同的大小。

In [10]:
x = torch.randn(4, 3)
y = torch.randn(4, 3)
z = torch.randn(1, 3)
cat_xyz = torch.cat([x, y, z], dim=0)
cat_xy = torch.cat([x, y], dim=1)
stack_xy0 = torch.stack([x, y], dim=0)
stack_xy1 = torch.stack([x, y], dim=1)
print(f'Shape of cat_xyz: {cat_xyz.shape}')
print(f'Shape of cat_xy: {cat_xy.shape}')
print(f'Shape of stack_xy0: {stack_xy0.shape}')
print(f'Shape of stack_xy1: {stack_xy1.shape}')

Shape of cat_xyz: torch.Size([9, 3])
Shape of cat_xy: torch.Size([4, 6])
Shape of stack_xy0: torch.Size([2, 4, 3])
Shape of stack_xy1: torch.Size([4, 2, 3])


- 转置：使用tensor.transpose()调整两个维度的顺序，使用tensor.permute()对多个维度的顺序进行调整。

In [11]:
x = torch.randn(2,3,4,5)
x_transpose = x.transpose(0, 2) # 输入要被交换的两个维度
x_permute = x.permute(1, 0, 2, 3) # 输入期待的维度顺序
print(f'Shape of x_transpose: {x_transpose.shape}')
print(f'Shape of x_permute: {x_permute.shape}')


Shape of x_transpose: torch.Size([4, 3, 2, 5])
Shape of x_permute: torch.Size([3, 2, 4, 5])


- 数学运算（四则运算）：tensor可以进行最基本的加法、减法、按位乘法、矩阵乘法等数学运算，参与运算的两个tensor需要满足一定的形状要求。

In [12]:
a = torch.randn(2, 3)
b = torch.randn(2, 3)
c = torch.randn(1, 3)
d = a + b 
e = a + c # 如果两个tensor的维度数量相同，其中一个tensor的某些维度长度为1，并且除了长度为1的维度外其他维度长度都相同，那么pytorch会自动对这些长度为1的维度进行broadcast。
print(f'Shape of a, b and a + b: {a.shape, b.shape, d.shape}')
print(f'Shape of a, c and a + c: {a.shape, c.shape, e.shape}')
f = d - e
print(f'Shape of d, e and d - e: {d.shape, e.shape, f.shape}')
g = f * e # 直接使用 * 号进行乘法表示对两个tensor进行按位乘。
print(f'Shape of f, e and f * e: {f.shape, e.shape, g.shape}')
h = torch.matmul(a, b.transpose(0, 1)) # 使用matmul进行矩阵乘法，tensor的形状要满足矩阵乘法的要求。
print(f'Shape of a, b and ab^T: {a.shape, b.shape, h.shape}')
a = a.unsqueeze(0).repeat(5, 1, 1)
b = b.unsqueeze(0).repeat(5, 1, 1)
i = torch.matmul(a, b.permute(0, 2, 1))
print(f'Shape of a, b and ab^T: {a.shape, b.shape, i.shape}') # 矩阵乘法可以批量计算，如果两个tensor维度数量相同，除了最后两维以外其他维度长度相等，并且最后两维满足矩阵乘法的要求，那么可以直接在最后两维进行矩阵乘法。

Shape of a, b and a + b: (torch.Size([2, 3]), torch.Size([2, 3]), torch.Size([2, 3]))
Shape of a, c and a + c: (torch.Size([2, 3]), torch.Size([1, 3]), torch.Size([2, 3]))
Shape of d, e and d - e: (torch.Size([2, 3]), torch.Size([2, 3]), torch.Size([2, 3]))
Shape of f, e and f * e: (torch.Size([2, 3]), torch.Size([2, 3]), torch.Size([2, 3]))
Shape of a, b and ab^T: (torch.Size([2, 3]), torch.Size([2, 3]), torch.Size([2, 2]))
Shape of a, b and ab^T: (torch.Size([5, 2, 3]), torch.Size([5, 2, 3]), torch.Size([5, 2, 2]))


- 数学运算（单点运算）：tensor可以进行求绝对值、对数、指数、平方、开方、三角函数、反三角函数、截断等丰富的数学运算。

In [ ]:
tensor = torch.rand(2, 3)
tensor_abs = tensor.abs()
tensor_abs = torch.abs(tensor) # 很多单点运算都有torch.xxx(tensor)和tensor.xxx()两种用法，二者功能完全一样。
tensor_log = torch.log(tensor)
tensor_log10 = torch.log10(tensor)
tensor_exp = torch.exp(tensor)
tensor_square = torch.square(tensor)
tensor_sqrt = torch.sqrt(tensor)
tensor_sin = torch.sin(tensor)
tensor_atan = torch.atan(tensor)
tensor_clamp = torch.clamp(tensor, min=0.2, max=0.8)

- 数学运算（统计量计算）：tensor的统计量包括最大、最小值（以及出现的位置），总和，平均值、中位数、众数、方差、标准差等。

In [17]:
tensor = torch.rand(2, 3)
tensor_max = torch.max(tensor) # 默认min、max操作只会返回最小、最大值
print(f'The maximum of tensor: {tensor_max}')
tensor_min, tensor_argmin = torch.min(tensor, dim=1) # 如果指定维度，那么min、max操作除了返回最小、最大值以外还会返回最小、最大值的位置。
print(f'The minimum of tensor along dim 1: {tensor_min}')
print(f'The argmin of tensor along dim 1: {tensor_argmin}')
tensor_sum = tensor.sum()
tensor_mean = tensor.mean()
tensor_mean = torch.mean(tensor) # 和单点操作类似，统计量计算操作通常也支持tensor.xxx()和torch.xxx(tensor)两种写法。
tensor_mean = tensor.mean(0, keepdim=True) # 对于沿某个维度的统计量计算而言，可以使用keepdim参数来控制返回值是否在该维度上补充一个长度为1的维度，来保持输入和输出的维度数量相同。
tensor_median = torch.median(tensor, dim=0, keepdim=False)
tensor_mode = torch.mode(tensor)
tensor_var = torch.var(tensor, unbiased=True) # 可以使用unbiased参数控制计算无偏或有偏的方差。
tensor_std = tensor.std()

The maximum of tensor: 0.9675918221473694
The minimum of tensor along dim 1: tensor([0.3212, 0.3916])
The argmin of tensor along dim 1: tensor([0, 1])


- 数学运算（比较运算）：tensor的常用比较运算包括比较大小，示性函数、排序等。

In [18]:
a = torch.randn(2, 3)
b = torch.randn(2, 3)
c = (a > b) # 直接使用>、<、>=、<=、==会得到按位比较的bool型结果。
d = torch.maximum(a, b)
e = torch.minimum(a, b) # 使用minimum或maximum会得到两个tensor每个位置的较小值或较大值。
f = torch.isnan(a)
g = torch.isinf(a)
h = torch.isfinite(a) # 示性函数返回tensor的每个位置是否为某种特殊类型，比如NaN、Inf等，返回值为bool型tensor。
i = torch.sort(a, dim=0)
j = torch.argsort(a, dim=1, descending=True) # 排序操作可以通过descending参数控制按从大到小或从小到大排序。
k = torch.topk(a, k=2, dim=1, largest=False, sorted=True) # topk操作可以用largest参数控制返回最大的tok或最小的topk，sorted参数控制返回值是否按大小顺序排序。
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)
print(g)
print(h)
print(i)
print(j)
print(k)

tensor([[ 0.8465, -1.5006,  0.5772],
        [ 0.3389, -0.1890,  0.8591]])
tensor([[ 0.9558,  1.4213, -0.0570],
        [ 1.3664, -2.2177,  1.2684]])
tensor([[False, False,  True],
        [False,  True, False]])
tensor([[ 0.9558,  1.4213,  0.5772],
        [ 1.3664, -0.1890,  1.2684]])
tensor([[ 0.8465, -1.5006, -0.0570],
        [ 0.3389, -2.2177,  0.8591]])
tensor([[False, False, False],
        [False, False, False]])
tensor([[False, False, False],
        [False, False, False]])
tensor([[True, True, True],
        [True, True, True]])
torch.return_types.sort(
values=tensor([[ 0.3389, -1.5006,  0.5772],
        [ 0.8465, -0.1890,  0.8591]]),
indices=tensor([[1, 0, 0],
        [0, 1, 1]]))
tensor([[0, 2, 1],
        [2, 0, 1]])
torch.return_types.topk(
values=tensor([[-1.5006,  0.5772],
        [-0.1890,  0.3389]]),
indices=tensor([[1, 2],
        [1, 0]]))


- 其他常用操作：包括复制（clone）、累加（cumsum）、累乘（cumprod）、爱因斯坦求和（einsum）、摊平（flatten、ravel）、范数计算（norm）等。

torch.einsum可以实现一对tensor的特定操作, 对于两个tensor: X和Y, 它们在某些维度上相同, 那么可以使用eisum简单地将这些相同的维度进行缩并
比如X: (i, m, j, n), Y: (m, o, p, n), 可以进行如下操作对m, n两个维度进行缩并:
$Z_{i,j,o,p} = \sum_m\sum_n X_{i,j,m,n}*Y_{m, o, p, n}$.

In [14]:
x = torch.randn(2, 3)
x_copy = x.clone()
x_cumsum = torch.cumsum(x, dim=1)
x_cumprod = torch.cumprod(x, dim=1)
X = torch.randn(2, 3, 4, 5)
Y = torch.randn(3, 6, 7, 5)
Z = torch.einsum('imjn,mopn->ijop', X, Y)
print(Z.shape)
Z_norm = torch.norm(Z, p=2, dim=-1)
Z = Z.flatten(start_dim=1, end_dim=2) # flatten操作可以对某段连续的维度进行摊平
print(Z.shape)
Z = Z.ravel() # ravel操作可以直接将tensor摊平成一维
print(Z.shape)

torch.Size([2, 4, 6, 7])
torch.Size([2, 24, 7])
torch.Size([336])


### Tensor自动求导机制

Pytorch中的tensor和Numpy的ndarray最大的区别就在于，使用tensor可以实现自动的导数计算。神经网络的本质是一个可以优化的函数，而在诸多优化方法中，最常见的一种就是基于梯度的优化。对于简单的函数形式，导数可以很容易地用解析式表示出来，然而神经网络作为一种高度复杂的函数，通常包含上千万个甚至上亿个参数，并且包含大量复杂的非线性操作，想要手动求得每个参数的导数根本不可能。Pytorch则可以帮助我们自动地对网络中所有的参数进行导数计算，

那么怎么样实现导数的自动计算呢？下面提供了一个简单的例子。

In [19]:
w1 = torch.randn(2, 3, requires_grad=True).float()
x = torch.tensor([[1.0],[2.0],[3.0]]).float()
b1 = torch.randn(2, 1, requires_grad=True).float()
y = torch.matmul(w1, x) + b1
w2 = torch.randn(1, 2, requires_grad=True).float()
z = torch.matmul(w2, y)
loss = z.mean()
loss.backward()

对标量loss执行backward操作后，就可以得到网络中所有参数（w1，b1，w2）的导数。

In [20]:
print('The grad of w1:')
print(w1.grad)
print('The grad of b:')
print(b1.grad)
print('The grad of w2:')
print(w2.grad)
print('The grad of y:')
print(y.grad)

The grad of w1:
tensor([[-0.0572, -0.1145, -0.1717],
        [ 0.0489,  0.0978,  0.1468]])
The grad of b:
tensor([[-0.0572],
        [ 0.0489]])
The grad of w2:
tensor([[-1.7164,  0.3696]])
The grad of y:
None


C:\Users\86182\AppData\Local\Temp\ipykernel_28808\1429542275.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:485.)
  print(y.grad)


我们成功获得了所有参数（w1，b1，w2）的导数，但当我们访问中间计算结果y的导数时，输出的结果为None，并且显示了一段warning信息。仔细阅读warning信息可以发现，Pytorch默认只会保留“叶子节点”的导数，而y是一个“非叶子节点”。什么是叶子节点，什么是非叶子节点呢？在神经网络的计算中，会有很多类似于y这样的中间结果，它们是通过对其他tensor进行计算或操作得到的，这样的tensor被称为非叶子节点。而w1，b1，w2这些参数都是直接创建得到的tensor，这些tensor则称为叶子节点。为了减少存储量，Pytorch只会默认保留叶子节点的梯度信息。

如果一定要获得非叶子节点的导数，那么则需要在backward操作之前，对非叶子节点执行retain_grad操作，例如进行如下操作后，就可以正常获取y的倒数了。

In [17]:
w1 = torch.randn(2, 3, requires_grad=True).float()
x = torch.tensor([[1.0],[2.0],[3.0]]).float()
b1 = torch.randn(2, 1, requires_grad=True).float()
y = torch.matmul(w1, x) + b1
y.retain_grad()
w2 = torch.randn(1, 2, requires_grad=True).float()
z = torch.matmul(w2, y)
loss = z.mean()
loss.backward()
print('The grad of y:')
print(y.grad)

The grad of y:
tensor([[-1.7572],
        [ 1.1631]])


可以通过访问.is_leaf属性来获知一个tensor是否是叶子节点。注意，如果对一个叶子节点进行了操作，也会将其变为非叶子节点。

In [21]:
w1 = torch.randn(3, 2, requires_grad=True).float()
print(w1.is_leaf)
w1 = w1.view(2, 3)
print(w1.is_leaf)
x = torch.tensor([[1.0],[2.0],[3.0]]).float()
print(x.is_leaf)
b1 = torch.randn(2, 1, requires_grad=True).float()
print(b1.is_leaf)
y = torch.matmul(w1, x) + b1
print(y.is_leaf)

True
False
True
True
False


如果直接修改一个叶子节点的值，则可能会直接获得报错信息（取决于Pytorch的版本，也可能会不显示报错信息而将叶子节点变为非叶子节点）。

In [22]:
w1 = torch.randn(3, 2, requires_grad=True).float()
print(w1.is_leaf)
w1[0, 1] = 0.0
print(w1.is_leaf)

True


RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.